In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

###  hyperparameter tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate= 0.001
dropout_rate= 0.5

input_shape = (32,32,3)
num_classes = 10

### preprocess

In [3]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train_dataset.csv')
train_df.head()

,train_path,name
0,cifar/train\0_frog.png,frog
1,cifar/train\10000_automobile.png,automobile
2,cifar/train\10001_frog.png,frog
3,cifar/train\10002_frog.png,frog
4,cifar/train\10003_ship.png,ship


In [6]:
test_df = pd.read_csv('test_dataset.csv')
test_df.head()

,test_path,name
0,cifar/test\0_cat.png,cat
1,cifar/test\1000_dog.png,dog
2,cifar/test\1001_airplane.png,airplane
3,cifar/test\1002_ship.png,ship
4,cifar/test\1003_deer.png,deer


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    width_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255.
)

In [8]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'train_path',
    y_col = 'name',
    target_size = input_shape[:2],
    batch_size = batch_size
)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'test_path',
    y_col = 'name',
    target_size = input_shape[:2],
    batch_size = batch_size
)


Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


### build model

In [10]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32,(3,3),padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs=net, name = 'Basic_CNN')

In [13]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [15]:
batch_size

32

In [16]:
len(train_df)/32

1562.5

In [17]:
len(train_generator)

1563

In [21]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs = num_epochs,
    validation_data = test_generator,
    validation_steps = len(test_generator)
)

Epoch 1/10
1563/1563 [==============================] - 166s 106ms/step - loss: 1.7419 - accuracy: 0.3602 - val_loss: 1.3308 - val_accuracy: 0.5054
Epoch 2/10
1563/1563 [==============================] - 146s 93ms/step - loss: 1.4446 - accuracy: 0.4749 - val_loss: 1.2950 - val_accuracy: 0.5239
Epoch 3/10
1563/1563 [==============================] - 146s 93ms/step - loss: 1.3493 - accuracy: 0.5140 - val_loss: 1.2085 - val_accuracy: 0.5566
Epoch 4/10
1563/1563 [==============================] - 148s 94ms/step - loss: 1.2945 - accuracy: 0.5333 - val_loss: 1.2104 - val_accuracy: 0.5683
Epoch 5/10
1563/1563 [==============================] - 147s 94ms/step - loss: 1.2541 - accuracy: 0.5503 - val_loss: 1.1016 - val_accuracy: 0.6014
Epoch 6/10
1563/1563 [==============================] - 146s 94ms/step - loss: 1.2237 - accuracy: 0.5630 - val_loss: 1.0372 - val_accuracy: 0.6195
Epoch 7/10
1563/1563 [==============================] - 147s 94ms/step - loss: 1.1961 - accuracy: 0.5712 - val_loss: 